<a href="https://colab.research.google.com/github/LetsTrie/COVID-19-BD-Data/blob/master/test_rates/TestRateClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from sklearn.linear_model import Ridge
from pandas import DataFrame
from numpy import *
import copy
import json, io
import sys
import math

In [24]:
dataset = pd.read_csv('data.csv')
df = dataset.copy()
datasetArray = df.values

# Dataset Column Number
COUNTRY_INDEX = 2
TEST_PER_THOUSAND_INDEX = 28

TOTAL_COUNT = {}
TOTAL_TEST_RATE = {}

for dataset in datasetArray:
    country_name = dataset[COUNTRY_INDEX]
    test_per_thousand = dataset[TEST_PER_THOUSAND_INDEX]
    if country_name not in TOTAL_COUNT: 
        TOTAL_COUNT[country_name] = 0
        TOTAL_TEST_RATE[country_name] = 0

    if not np.isnan(test_per_thousand): 
        TOTAL_COUNT[country_name] += 1
        TOTAL_TEST_RATE[country_name] += test_per_thousand

# Simple Average
avg_test_rate = {}
for key in TOTAL_TEST_RATE:
    avg_test_rate[key] = 0
    if TOTAL_COUNT[key] != 0:
        avg_test_rate[key] = TOTAL_TEST_RATE[key] / TOTAL_COUNT[key]

print(avg_test_rate)

{'Afghanistan': 0, 'Albania': 0, 'Algeria': 0, 'Andorra': 0, 'Angola': 0, 'Antigua and Barbuda': 0, 'Argentina': 0, 'Armenia': 0, 'Australia': 1.521058201058202, 'Austria': 1.2614188679245288, 'Azerbaijan': 0, 'Bahamas': 0, 'Bahrain': 5.029130434782608, 'Bangladesh': 0.06268592057761738, 'Barbados': 0, 'Belarus': 0.9838823529411763, 'Belgium': 1.8813785714285711, 'Belize': 0, 'Benin': 0, 'Bhutan': 0, 'Bolivia': 0.12624793388429753, 'Bosnia and Herzegovina': 0, 'Botswana': 0, 'Brazil': 0, 'Brunei': 0, 'Bulgaria': 0.6881711229946527, 'Burkina Faso': 0, 'Burundi': 0, 'Cambodia': 0, 'Cameroon': 0, 'Canada': 1.183432330827068, 'Cape Verde': 0, 'Central African Republic': 0, 'Chad': 0, 'Chile': 1.1255836575875484, 'China': 0, 'Colombia': 0.37531884057971004, 'Comoros': 0, 'Congo': 0, 'Costa Rica': 0.2172146118721461, "Cote d'Ivoire": 0.035951327433628305, 'Croatia': 0.7424312977099241, 'Cuba': 0.40243023255813937, 'Cyprus': 2.8629802371541513, 'Czech Republic': 0.9864105960264905, 'Democrati

In [ ]:
inputSet = []
testRates = []

for dataset in datasetArray:
    x = []
    for i in range(1, len(dataset)-1):
        x.append(dataset[i])
    inputSet.append(x)
    testRates.append(dataset[len(dataset) - 1])
print(1)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(inputSet, testRates, test_size=0.2)
print(len(X_train))

12


In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(inputSet, testRates)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
bd_data = [[165379406], [1265], [4570], [2703.763], [0.8]]
print(model.predict(bd_data)[0])
bd_test_rate = (model.predict(bd_data)[0] * 100) / 100000
print(bd_test_rate)

#required rate = 2.885 %
#currrent rate = 1.518 %
#tests per thousand = 0.106, % = 0.0106
#positive per million = 11.215, % = 0.000112
#current test positivity rate = 10.6%

2885
2.885


In [36]:
main_data = pd.read_csv('owid-covid-data.csv')
main_data = main_data.fillna(method='ffill')
main_data = main_data.fillna(method='bfill')
main_data_frame = main_data.copy()
main_data_frame = main_data_frame.values

In [43]:
# 10, 27, 35, 36, 40, 47

countries = set([])
for i in range(len(main_data_frame)):
  countries.add(main_data_frame[i][2])

country_data = []

def filter(country):
  data = []
  for i in range(len(main_data_frame)):
    if main_data_frame[i][2] == country:
      row = main_data_frame[i]
      dropped_row = [row[35], row[36], row[40], row[35]*row[40], row[10], row[47], row[28]]
      data.append(dropped_row)

  return data[len(data)-1]

for country in countries:
  selected_countries = filter(country)
  country_data.append(selected_countries)

In [44]:
def form_param_array(param_no):
  dataset = []
  for i in range(len(country_data)):
     dataset.append(country_data[i][param_no])
  
  return dataset

def form_dataset(low, high, param_no):
  dataset = []
  for country in country_data:
    if low <= country[param_no] <= high:
      dataset.append([country[param_no], country[6]])
  return dataset

def mean_deviation(dataset):
  mean = np.mean(dataset)
  mean_sum = 0
  for n in dataset:
    mean_sum += abs(mean - n)

  return mean_sum/len(dataset)

In [45]:
def weighted_mean(data, bd_data):
  weights = []
  avg = 0
  print(len(data))

  if len(data) == 0 or np.isnan(data).any():
    return 0

  for i in range(len(data)):
    weights.append(1 / abs(bd_data - data[i][0]))
  _max = np.max(weights)
  norm_weights = list(map(lambda x: x/_max, weights)) 
  for i in range(len(data)):
    avg += data[i][1]*norm_weights[i]
  avg = avg / np.sum(norm_weights)

  return avg

In [50]:
bd_data = [165379406, 1265, 4570, 2740000000, 2703.763, 0.8]
allowed_std = 1
AVGs = []
# print(np.mean(form_param_array(6)))
for i in range(0, 6):
  param_array = form_param_array(i)
  mean_dev = mean_deviation(param_array)
  low = abs(bd_data[i] - allowed_std*mean_dev)
  high = bd_data[i] + allowed_std*mean_dev
  dataset = form_dataset(low, high, i)
  avg = weighted_mean(dataset, bd_data[i])
  AVGs.append(avg)

# print(AVGs)
  